# Titanic Tweak
Målet med denna notebook är att försöka skaffa bra noggrannhet för ett beslutsträd med data från Titanic.

#### Importerar bibliotek

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


#### Läser in data

In [102]:
train_titanic_data = pd.read_csv('train_titanic.csv')


#### Grundläggande överblick av data

In [103]:
train_titanic_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [104]:
train_titanic_data.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [105]:
train_titanic_data.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

#### Förbehandling av data
Vi börjar med att titta på vilka attribut som kan sållas bort då de antagligen har liten eller låg påverkan på en passagerares överlevnad i katastrofen. Vi väljer att behålla följande:
* Survived (vårat facit)
* Pclass (Speglar till viss del placeringen på skeppet)
* Sex och Age (Efter uttrycket "Kvinnor och barn först")
* Fare (Likt Pclass kan detta möjligtvis spegla placeringen på skeppet).


In [106]:
ttd_sliced = train_titanic_data[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]

#### Age NULL
När vi tittar på vår nya datadel kan vi se att vi har vissa passagerare för vilka det saknas ålder. Vi skapar tre versioner av datan: En utan dessa passagerare, en där de får genomsnittsåldern för alla andra passagerare, och en för alla andra passagerare som delar Pclass och Sex med dem.

In [107]:
ttd_sliced.count()

Survived    891
Pclass      891
Sex         891
Age         714
Fare        891
dtype: int64

In [120]:
ttd_sliced_short = ttd_sliced.loc[~ttd_sliced['Age'].isnull()]

ttd_sliced_ship = ttd_sliced[:]
ttd_sliced_ship['Age'] = ttd_sliced_ship['Age'].fillna(ttd_sliced['Age'].mean())

ttd_sliced_demo = ttd_sliced[:]
for sex in ttd_sliced_demo['Sex'].unique():
    print(sex)
    for p_class in ttd_sliced_demo['Pclass'].unique():
        print(p_class)
        mean_age = ttd_sliced.loc[ttd_sliced['Sex'] == sex, ttd_sliced['Pclass'] == p_class]
        print(mean_age)
        ttd_sliced_demo.loc[ttd_sliced_demo['Sex'] == sex, ttd_sliced_demo['Pclass'] == p_class]['Age'] = mean_age


male
3
     Survived  Pclass   Sex   Age     Fare
0           0       3  male  22.0   7.2500
4           0       3  male  35.0   8.0500
5           0       3  male   NaN   8.4583
6           0       1  male  54.0  51.8625
7           0       3  male   2.0  21.0750
..        ...     ...   ...   ...      ...
883         0       2  male  28.0  10.5000
884         0       3  male  25.0   7.0500
886         0       2  male  27.0  13.0000
889         1       1  male  26.0  30.0000
890         0       3  male  32.0   7.7500

[577 rows x 5 columns]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
ttd_sliced_demo

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500
...,...,...,...,...,...
886,0,2,male,27.0,13.0000
887,1,1,female,19.0,30.0000
888,0,3,female,NaN,23.4500
889,1,1,male,26.0,30.0000
